In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint

def scrape_job_details(url):
    """Scrape detailed information from a single job URL"""
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Helper function for safe extraction
        def get_text(selector, class_name):
            element = selector.find(class_=class_name)
            return element.get_text(strip=True) if element else ""
        
        return {
            "title": get_text(soup, "job-ad-display-2a5cm7"),
            "company": get_text(soup, "job-ad-display-du9bhi"),
            "location": get_text(soup, "at-listing__list-icons_location map-trigger job-ad-display-1drtjkr"),
            "job_type": get_text(soup, "at-listing__list-icons_contract-type job-ad-display-1drtjkr"),
            "employment_type": get_text(soup, "at-listing__list-icons_work-type job-ad-display-1drtjkr"),
            "description": get_text(soup, "job-ad-display-nfizss"),
            "requirements": get_text(soup, "at-section-text-profile-content"),
            #"salary": get_text(soup, "at-listing__list-icons_salary job-ad-display-1sx5ten"),

            
        }
        
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_jobs_dataframe(pages=1):
    """Main function that returns a DataFrame of all jobs"""
    base_url = "https://www.stepstone.de/jobs/data-science?page="
    job_url = "https://www.stepstone.de"
    all_jobs = []
    
    for page in range(151, pages + 1):
        print(f"Scraping page {page}...")
        try:
            response = requests.get(
                base_url + str(page),
                headers={
                    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
                },
                timeout=10
            )
            soup = BeautifulSoup(response.content, "html.parser")
            job_cards = soup.find_all("article", class_="res-4cwuay")

            if not job_cards:
                print("No job cards found on this page.")
                continue

            for card in job_cards:
                try:
                    relative_url = card.find("a", class_="res-1fudl87")["href"]
                    url = job_url + relative_url if relative_url.startswith("/") else relative_url
                    
                    job_data = scrape_job_details(url)
                    if job_data:
                        all_jobs.append(job_data)
                    
                except Exception as e:
                    print(f"Error processing job card: {str(e)}")
                    continue
                    
        except Exception as e:
            print(f"Error on page {page}: {str(e)}")
            continue
            
    return pd.DataFrame(all_jobs) if all_jobs else pd.DataFrame()

# Usage example:
df = get_jobs_dataframe(pages=190)  
if not df.empty:
    print(f"Successfully scraped {len(df)} jobs")
    print(df.head())
    df.to_csv('stepstone_jobs[151-190].csv', index=False) 
else:
    print("No jobs were scraped")

Scraping page 151...
Scraping page 152...
Scraping page 153...
Scraping page 154...
Scraping page 155...
Scraping page 156...
Scraping page 157...
Scraping page 158...
Scraping page 159...
Scraping page 160...
Scraping page 161...
Scraping page 162...
Scraping page 163...
Scraping page 164...
Scraping page 165...
Scraping page 166...
Scraping page 167...
Scraping page 168...
Scraping page 169...
Scraping page 170...
Scraping page 171...
Scraping page 172...
Scraping page 173...
Scraping page 174...
Scraping page 175...
Scraping page 176...
Scraping page 177...
Scraping page 178...
Scraping page 179...
Scraping page 180...
Scraping page 181...
Scraping page 182...
Scraping page 183...
Scraping page 184...
No job cards found on this page.
Scraping page 185...
No job cards found on this page.
Scraping page 186...
No job cards found on this page.
Scraping page 187...
No job cards found on this page.
Scraping page 188...
No job cards found on this page.
Scraping page 189...
No job cards fou

In [19]:
pd.read_csv("stepstone_jobs[151-190].csv")


,title,company,location,job_type,employment_type,description,requirements
0,Anticipatory Action Expert,Deutsche Welthungerhilfe e.V.,Bonn,Befristeter Vertrag,Vollzeit,Are you looking for a meaningful job where you...,A university degree in a relevant field like d...
1,SENIOR SOLUTION ARCHITECT IN DER KONZERNKONSOL...,concellens GmbH,Potsdam,Feste Anstellung,"Homeoffice möglich, Vollzeit",Unterstützung von fachlicher Konzeption und Um...,Solide Kenntnisse im Bereich der Wirtschaftsin...
2,Electrical engineer,Menger Engineering GmbH,"Leipzig, Germany",Feste Anstellung,Vollzeit,We are a medium-sized company based in Leipzig...,"a qualification (engineer, master craftsman, t..."
3,Intern AI & Management Consulting (Value Engin...,Celonis SE,München,"Studentenjobs, Werkstudent","Homeoffice möglich, Teilzeit","We're Celonis, the global leader in Process Mi...",Student of Business Administration / Industria...
4,Internship Influencer Marketing France (m/f/x),Hanfgeflüster GmbH,Berlin,Praktikum,Teilzeit,Are you looking for an exciting challenge in a...,You are currently studying or have completed a...
...,...,...,...,...,...,...,...
804,IT-Specialist am Standort Norderstedt (m/w/d),FERCHAU – Connecting People and Technologies,Norderstedt,Feste Anstellung,"Homeoffice möglich, Vollzeit","Menschen und Technologien zu verbinden, den Pe...",Womit du uns überzeugstAbgeschlossene Ausbildu...
805,Projektkoordinator Formulierungs- und Prozesse...,FERCHAU – Connecting People and Technologies,Biberach an der Riß,Feste Anstellung,Vollzeit,Du möchtest deine Karriereentwicklung nicht de...,Das bringst du mitAbgeschlossenes Bachelor-/Ma...
806,Quereinsteiger Produktion Pharma (m/w/d),FERCHAU – Connecting People and Technologies,Biberach an der Riß,Feste Anstellung,Vollzeit,Du möchtest deine Karriereentwicklung nicht de...,Das bringst du mitAbgeschlossene Ausbildung al...
807,Laborant / CTA / BTA (m/w/d),FERCHAU – Connecting People and Technologies,Biberach an der Riß,Feste Anstellung,Vollzeit,Die besten Köpfe für die unterschiedlichsten A...,Das bringst du mitAbgeschlossene Ausbildung al...


In [ ]:
url = 'https://www.stepstone.de/stellenangebote--Procurement-Specialist-m-w-d-Data-Science-AI-Stuttgart-Leadec-Management-Central-Europe-BV-Co-KG--12811449-inline.html'
response = requests.get(url)
print(response)